# [Uninformed Search Algorithms](00_main_report.ipynb)

The informed search algorithms are a class of algorithms that use problem-specific knowledge to guide the search. 

$$ f(n) = g(n) + h(n) $$

where $f(n)$ is the total cost of the node, $g(n)$ is the cost to reach the node, and $h(n)$ is the heuristic cost of the node.

```mermaid
graph LR
A((Start)) --> B((Goal))
A --> D((D))
A --> E((E))
D --> F((F))
E --> F
F --> B
```

In this notebook, we will implement the following informed search algorithms:

- Depth First Search (DFS)
    - Depth Limited Search (DLS)
    - Iterative Deepening Search (IDS)
    - Recursive Depth Limited Search (RDLS)
- Breadth First Search (BFS)
- Uniform Cost Search  (UCS)

In [1]:
from collections import deque
from queue import Queue
from timeit import timeit  # for timing the algorithms

import networkx as nx  # for visualizing the graphs

G = {  # test graph
    "A": ["B", "C"],
    "B": ["A", "D", "E"],
    "C": ["A", "F"],
    "D": ["B"],
    "E": ["B", "F"],
    "F": ["C", "E"],
}

# ? BENCHMARK FUNCTION --------------------------------------------------------------------------------------------------------------------------------------------------------

def benchmark_function(name: str, iter: int=10000) -> None:
    setup = f"from __main__ import G, {name}"  # setup the function
    res = timeit(f"{name}(G, 'A')", setup=setup, number=iter)  # time the function
    print(f"{name:<25} finished {iter} runs in {res:.5f} seconds")  # print the results

```mermaid
graph LR
A --> B
A --> C

B --> A
B --> D
B --> E

C --> A
C --> F

D --> B

E --> B
E --> F

F --> C
F --> E
```

### [Depth First Search (DFS)](https://en.wikipedia.org/wiki/Depth-first_search)

Is an algorithm for **traversing or searching tree or graph data structures**. The algorithm starts at the root node (selecting some arbitrary node as the root node in the case of a graph) and explores as far as possible along each branch before backtracking.

- $ Complexity: O(b^d) $
- $ Space: O(bd) $

where $b$ is the branching factor and $d$ is the depth of the goal node.

![dfs_01](./../resources/img/dfs_vs_bfs.gif)

In [20]:
def depth_first_search(graph: dict, start: str) -> set:
    """
    Implementation of depth first search using collection.deque.
    
    ### Differences from BFS:
    1) pop last element instead of first one
    2) add adjacent elements to stack without exploring them
    3) add explored nodes to explored set
    4) return explored set instead of solution
    """
    explored: set = set(start)  # set of explored nodes
    stack: list = [start]  # use list as a stack

    while stack:  # while stack is not empty
        v = stack.pop()  # pop last element
        explored.add(v)  # add to explored nodes
        for adj in reversed(graph[v]):  # for each adjacent node
            if adj not in explored:  # if not explored
                stack.append(adj)  # add to stack
    return explored  # return the explored nodes

#### Depth Limited Search (DLS)

Is a variant of depth-first search that limits the depth of the search tree. It is a recursive algorithm that uses a depth limit $l$ as a stopping condition. If the depth of the node exceeds the limit, the search is terminated.

In [8]:
def depth_limited_search(graph: dict, start: str, limit: int = 2) -> list[str]:
    """
    Depth Limited Search algorithm
    """
    visited = {start}  # set of visited nodes
    result = [start]  # list of visited nodes
    stack = [start]  # stack of visited nodes
    while stack:  # while stack is not empty
        v = stack.pop()  # get the last node from the stack
        if len(result) > limit:  # if the limit is reached
            return result  # return the result list (visited nodes)
        for child in graph[v]:  # for each child of the node
            if child not in visited:  # if child is not visited
                visited.add(child)  # mark child as visited
                result.append(child)  # add child to the result
                stack.append(child)  # add child to the stack
    return result  # return the result list (visited nodes)

##### Iterative (IDS or IDDFS)

Is a state space/graph search strategy in which a depth-limited version of depth-first search is run repeatedly with increasing depth limits until the goal is found.

IDDFS is optimal like breadth-first search, but **uses much less memory; at each iteration**, it visits the nodes in the search tree in the same order as depth-first search, but the cumulative order in which nodes are first visited is effectively breadth-firs

In [16]:
def iterative_deepening_search(graph: dict, start: str, goal: str = 'D') -> list[str]:
    """
    Iterative Deepening Search algorithm
    """
    for depth in range(0, len(graph)):  # for each depth
        result = depth_limited_search(graph, start, depth)  # search for the goal
        if result[-1] == goal:  # if the goal is found
            return result  # return the result list (visited nodes)
    return []  # return empty list if the goal is not found


#### Recursive Depth Limited Search (RDLS)

Is a variant of depth-first search that limits the depth of the search tree. It is a recursive algorithm that uses a depth limit $l$ as a stopping condition. If the depth of the node exceeds the limit, the search is terminated.

In [19]:
def recursive_dls(graph: dict, start: str, goal: str, limit: int) -> list[str]:
    """
    Recursive Depth Limited Search algorithm
    """
    visited = {start}  # set of visited nodes
    result = [start]  # list of visited nodes
    if start == goal:  # if the goal is found
        return result  # return the result list (visited nodes)
    elif limit == 0:  # if the limit is reached
        return []  # return empty list
    else:
        for child in graph[start]:  # for each child of the node
            if child not in visited:  # if child is not visited
                visited.add(child)  # mark child as visited
                result.append(child)  # add child to the result
                result = recursive_dls(graph, child, goal, limit - 1)  # search for the goal
                if result:  # if the goal is found
                    return result  # return the result list (visited nodes)
    return []  # return empty list if the goal is not found

### [Breadth First Search (BFS)](https://en.wikipedia.org/wiki/Breadth-first_search)

Is an algorithm for **traversing or searching tree or graph data structures**.

It starts at the tree root (or some arbitrary node of a graph, sometimes referred to as a 'search key'[1]), and explores all of the neighbor nodes at the present depth prior to moving on to the nodes at the next depth level.

- Complexity: $ O(V+E) $
- Space: $ O(V) $

where $V$ is the number of vertices and $E$ is the number of edges.

![bfs_01](./../resources/img/bfs.gif)

In [8]:
def breadth_first_search(graph: dict, start: str) -> list[str]:
    """
    Breadth First Search algorithm
    """
    visited = {start}  # set of visited nodes
    result = [start]  # list of visited nodes
    queue = Queue()  # queue of visited nodes
    queue.put(start)  # add start node to the queue
    while not queue.empty():  # while queue is not empty
        v = queue.get()  # get the first node from the queue
        for child in graph[v]:  # for each child of the node
            if child not in visited:  # if child is not visited
                visited.add(child)  # mark child as visited
                result.append(child)  # add child to the result
                queue.put(child)  # add child to the queue
    return result  # return the result list (visited nodes)

### [Uniform Cost Search (UCS)](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm)

This algorithm is very similar to Dijkstra's algorithm for finding the shortest path between nodes in a graph, but with a different **evaluation function** ("Dijsktra for large graphs...").

The UCS algorithm is a **greedy algorithm** that finds a **minimum spanning tree** for a weighted undirected graph.

Always evaluates the node with the **lowest path cost**.

- Complexity: $ O(b^{1+\frac{C*}{\epsilon}}) $
- Space: $ O(b^{1+\frac{C*}{\epsilon}}) $

where $b$ is the branching factor, $C*$ is the cost of the optimal solution, and $\epsilon$ is the accuracy of the heuristic.

![ucs_01](./../resources/img/ucs.gif)

In [9]:
def uniform_cost_search(self, initial_state) -> None:
    """
    Uniform Cost Search algorithm

    This algorithm is a special case of the A* algorithm
    
    """
    pass

#### Dijkstra's Algorithm

Is an algorithm for finding the shortest paths between nodes in a graph, which may represent, for example, road networks. It was conceived by computer scientist Edsger W. Dijkstra in 1956 and published three years later.

- Complexity: $ O(V^2) $
- Space: $ O(V) $

where $V$ is the number of vertices.

![dijkstra](./../resources/img/dijkstra.gif)

----
# Test It


- [some animation comparason](https://www3.cs.stonybrook.edu/~skiena/combinatorica/animations/search.html)

In [23]:
if __name__ == "__main__":
    import doctest  # for testing the functions
    doctest.testmod()  # run doctests

    benchmark_function("depth_first_search", iter=10000)
    benchmark_function("depth_first_search", iter=100000)

    benchmark_function("depth_limited_search", iter=10000)
    benchmark_function("depth_limited_search", iter=100000)

    benchmark_function("iterative_deepening_search", iter=10000)
    benchmark_function("iterative_deepening_search", iter=100000)

    benchmark_function("breadth_first_search", iter=10000)
    benchmark_function("breadth_first_search", iter=100000)

    # benchmark_function("uniform_cost_search")
    
    # benchmark_function("dijkstra")
    

depth_first_search        finished 10000 runs in 0.02551 seconds
depth_first_search        finished 100000 runs in 0.20951 seconds
depth_limited_search      finished 10000 runs in 0.00595 seconds
depth_limited_search      finished 100000 runs in 0.05862 seconds
iterative_deepening_search finished 10000 runs in 0.05646 seconds
iterative_deepening_search finished 100000 runs in 0.60497 seconds
breadth_first_search      finished 10000 runs in 0.01538 seconds
breadth_first_search      finished 100000 runs in 0.16069 seconds
